This jupyter notebook will focus on creating dashboard for Australia wildfire dataset with following componenets:  
1. Select region
2. Select year
3. Charts: Pie Chart to display Monthly average Estimated fire area for selected Regions in selected year
4. Bar chart to display Monthly average count of pixels for presumed vegetation fire for selected regions in selected year

Requirments to create the expected result:  
1. A dropdowm menu for selecting year
2. A radioitem for choosing year
3. The layout will be designed as follows:
4. An outer division with two inner divisions
5. One of the inner division will have information about radio item and dropdown and the other one is for adding graphs
6. Callback function to Compute data, create graph and return to layout

In [44]:
# FIRST STEP:  Import the required libraries and read the dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import datetime as dt
data = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv"
df = pd.read_csv(data)

df['Month'] = pd.to_datetime(df['Date']).dt.month_name() #used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year
df.head()

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced,Month,Year
0,NSW,1/4/2005,8.68000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R,January,2005
1,NSW,1/5/2005,16.61125,322.475000,62.362500,85.500000,8.088793,65.428571,8,R,January,2005
2,NSW,1/6/2005,5.52000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R,January,2005
3,NSW,1/7/2005,6.26400,313.870000,33.800000,92.200000,7.529940,56.700000,5,R,January,2005
4,NSW,1/8/2005,5.40000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R,January,2005


In [59]:
# STEP 2 Create an application layout
app = dash.Dash(__name__)

app.layout = html.Div(
    children =[
        #Step 3 Add title to the layout
        html.H1('Autralia Wildfire Dashboard', style={'text-align':'center','color':'pink','size': 40}),
        #Step 4 First Inner division adding dropdowns and other text selecting options
        html.Div(
            [ html.H2('Select Region'),
            dcc.RadioItems(
                [
                    {"label":"New South Wales","value": "NSW"},
                    {'label':'Northern Territories','value':'NT'},
                    {'label':'Queensland','value':'QL'},
                    {'label':'South Australia','value':'SA'},
                    {'label':'Tasmania','value':'TA'},
                    {'label':'Victoria','value':'VI'},
                    {'label':'Western Australia', 'value':'WA'}
                    
                        
        
                ],
            value='NSW',id='region', inline=True
        )]
        ),
        html.Div(
            [
               html.H2('Select Year' , style={'margin-right': '2em'}),
               dcc.Dropdown(df.Year.unique(),value=2005,id='year') 
            ]
        ),
        # Step 5 Second inner division for adding 2 output graphs
       html.Div(
           [
               html.Div([], id ='plot1'),
               html.Div([],id='plot2')
            
           ],
           style={}
       ) 
    
    ]
)

#STEP 5 Add the output and input components inside app.callback decorator
@app.callback([Output(component_id='plot1',component_property='children'),
               Output(component_id='plot2', component_property='children')],
              [Input(component_id='region', component_property='value'),
               Input(component_id='year',component_property='value')])

#STEP 6 is to add callback function
# Whenever an input property changes, the function wrapped by callback decorator is called automatically
# In this case we will define a function called reg_year_display which will be wrapped by our callback decorator
# The function first filters our dataframe df by selected value of region from radio items and year from dropdown as follows
# region_data = df[df['Region']] == input_region
#y_r_data = region_data[region_data['Year']==input_year]

def reg_year_display(input_region, input_year):
    region_data = df[df['Region'] == input_region]
    y_r_data = region_data[region_data['Year']== input_year]
    # Plot 1 : MONTHLY AVERAGE ESTIMATED FIRE AREA
    est_data = y_r_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(est_data,values='Estimated_fire_area', names='Month', title="{} : Monthly Average Estimated Fire Area in year {}".format(input_region,input_year))
    # Plot 2: Monthly average count of pixels for presumed vegetation fire
    veg_data = y_r_data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(veg_data,x='Month', y='Count', title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(input_region,input_year))

    return[dcc.Graph(figure=fig1),
          dcc.Graph(figure=fig2)]
    
    





if __name__ == '__main__':
    app.run_server(port=90)